In [1]:
import os
import rasterio
from rasterio.warp import transform_bounds
import overpy
import cv2
import numpy as np
from PIL import Image
import json

## This script is used to filter those tif images under all_train_cal, because we only want those images with towers.

In [2]:
def in_image(row,col,row_max,col_max):
    if row >= 256  and col >= 256 and row <= row_max - 256 and col <= col_max - 256 :
        return True

def write_geojson(data,valid_nodes,out_filename):
    geojson = {
        "type": "FeatureCollection",
        "features": []
    }
    for num in range(len(data.nodes)):
        if valid_nodes[num]:    
            node = data.nodes[num]
            feature = {
                "type": "Feature",
                "properties": {
                    "osm_id": node.id,
                    "name": node.tags.get("name"),
                    "barrier": node.tags.get("barrier"),
                    "highway": node.tags.get("highway"), 
                    "ref": node.tags.get("ref"),
                    "address": node.tags.get("address"),
                    "is_in": node.tags.get("is_in"),
                    "place": node.tags.get("place"),
                    "man_made": node.tags.get("man_made"),
                    "other_tags": "\"power\"=>\"tower\""
                },
                "geometry": {
                    "type": "Point",
                    "coordinates": [float(node.lon), float(node.lat)]
                }
            }
            geojson["features"].append(feature)
    
    # Write the GeoJSON to a file
    with open(out_filename, "w") as f:
        json.dump(geojson, f)

    print("Valid_GeoJSON file created successfully!")
    
def crop_square_image(img, xy, size=256):
    img = Image.fromarray(img)
    left = xy[0] - size
    top = xy[1] - size
    right = xy[0] + size
    bottom = xy[1] + size
    cropped_img = img.crop((left, top, right, bottom))    
    return cropped_img

def read_RGB_from_geotif_dis_plants_sands(src):  
    if src.count == 8:
        r = src.read(4)
        g = src.read(6)
        b = src.read(8)
    else:
        r = src.read(1)
        g = src.read(2)
        b = src.read(3)
    img = np.stack((r, g, b), axis=-1)
    img_uint8 = (img / img.max() * 255).astype(np.uint8)
    return img_uint8

In [3]:
img_folder = "/media/HD_Matcha_2/Yan/all_train_val/"
image_files = [file for file in sorted(os.listdir(img_folder)) if file.endswith(".tif")]
# save file name list into a .txt for future use, like matching the num with file name.
api = overpy.Overpass()
image_points = []
image_points_xy = []
for num in range(len(image_files)):
    image_name = f"{img_folder}{image_files[num]}"
    tiff_image = rasterio.open(image_name)    
    bounds = tiff_image.bounds
    if "metre" in tiff_image.crs.to_string():
        print("The Unit is metre. coordinate are transfered into lat-long in degrees.")
        bounds = transform_bounds(tiff_image.crs, "WGS84",
                                         bounds.left, bounds.bottom, bounds.right, bounds.top)
    else:
        print("The Unit is degree.")
    query_var = f"""[out:json];node({bounds[1]}, {bounds[0]} , {bounds[3]}, {bounds[2]})["power"="tower"];(._;>;);out;"""
    result = api.query(query_var)
    points = result.nodes       
    if len(points) != 0:
        valid_nodes = []
        new_result = type(result)()
        length_result_nodes = len(points)
        meta = tiff_image.meta
        col_max = meta['width'] ; row_max = meta['height']
        for p in range(len(points)):
            x_coord =  float(points[p].lon) ; y_coord = float( points[p].lat)
            row, col = rasterio.transform.rowcol(meta['transform'], xs=x_coord, ys=y_coord)
            if in_image(row,col,row_max,col_max): 
                valid_nodes.append([1])
                img=read_RGB_from_geotif_dis_plants_sands(tiff_image)
                image_points.append((x_coord,y_coord))
                image_points_xy.append((row,col))
                cropped_img = crop_square_image(img, [row,col], size=256)
                imgname = "cropped/" + image_files[num][:-4] + "_" + str(p) + ".jpg"
                cropped_img.save(imgname)  
            else:
                valid_nodes.append([0])
        length_valid_nodes = len(valid_nodes)
        with open('nodes_lengths.txt', 'a') as f:
            f.write(f"{length_result_nodes}\t{length_valid_nodes}\n")
        filename = "geojson/" + image_files[num][:-4] + ".geojson"
        write_geojson(result,valid_nodes,filename)
    else:
        os.remove(f"{img_folder}{image_files[num]}")
        print(f"No tower found, file deleted: {image_files[num]}")

ERROR 1: PROJ: proj_identify: /home/irene-oceanus/anaconda3/envs/yolo/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


The Unit is degree.
Valid_GeoJSON file created successfully!
The Unit is degree.


ERROR 1: PROJ: proj_identify: /home/irene-oceanus/anaconda3/envs/yolo/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


Valid_GeoJSON file created successfully!
The Unit is degree.


ERROR 1: PROJ: proj_identify: /home/irene-oceanus/anaconda3/envs/yolo/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


...

Valid_GeoJSON file created successfully!
The Unit is degree.


ERROR 1: PROJ: proj_identify: /home/irene-oceanus/anaconda3/envs/yolo/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


Valid_GeoJSON file created successfully!
The Unit is degree.


ERROR 1: PROJ: proj_identify: /home/irene-oceanus/anaconda3/envs/yolo/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


Valid_GeoJSON file created successfully!
